# 影像處理

處理某些影像顛倒問題：https://stackoverflow.com/questions/13872331/rotating-an-image-with-orientation-specified-in-exif-using-python-without-pil-in

In [17]:
def orientationCorrect(imgname):
    try:
        image=Image.open(imgname)
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation]=='Orientation':
                break
        exif=dict(image._getexif().items())

        if exif[orientation] == 3:
            image=image.rotate(180, expand=True)
        elif exif[orientation] == 6:
            image=image.rotate(270, expand=True)
        elif exif[orientation] == 8:
            image=image.rotate(90, expand=True)
        image.save(imgname)
        image.close()
    #遇到error跳過
    except Exception: #(AttributeError, KeyError, IndexError):
        # cases: image don't have getexif
        pass

In [16]:
import cv2
import os
from PIL import Image, ExifTags
origin_dir = "./pred/"
save_dir = "./data/pred/"

face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
emotion_name = ['happy','shocked','bored', 'frustrated', 'happy', 'confused','neutral','focus']
for e in emotion_name:
    face_num = 1
    for i in range(1,20):
        imgname= origin_dir +e+"/"+ e + " (" +str(i)+").JPG" #路徑不能為中文
        orientationCorrect(imgname)
        img = cv2.imread(imgname,0)#, cv2.IMREAD_COLOR)
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(img, scaleFactor = 1.3, minNeighbors = 5, minSize=(20,20))
        for (x,y,w,h) in faces:
            cv2.rectangle(img, (x-50,y-50), (x+w+50, y+h+50), (255,0,0),2)
            filename = e+str(face_num)+".jpg"
            image = Image.open(imgname).convert("L")#抓圖處
            imageCrop = image.crop((x-100,y-100,x+w+100,y+h+100))#裁切圖片
            imageResize = imageCrop.resize((250,250),Image.ANTIALIAS)#高品質
            imageResize.save(save_dir+e+'/'+filename)
            face_num += 1